Import Libraries

In [3]:
import pandas as pd
import sqlalchemy 
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
import pyodbc

## using SQLalchemy



In [5]:
# Using sqlalchemy
server = 'DABAS\SQLEXPRESS'
database = 'ChurnDataset'
driver = 'SQL Server'
connection = f"mssql://@{server}/{database}?driver={'SQL Server'}"
engine = create_engine(connection)
connect = engine.connect()


In [11]:
#read data and covert to df
df = pd.read_sql(
      'SELECT * FROM Data',
  engine,
  index_col='customer_id')
df

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
customer_id,,,,,,,,,,,
15634602,619,France,Female,42,2,0.000000,1,True,True,101348.882812,True
15647311,608,Spain,Female,41,1,83807.859375,1,False,True,112542.578125,False
15619304,502,France,Female,42,8,159660.796875,3,True,False,113931.570312,True
15701354,699,France,Female,39,1,0.000000,2,False,False,93826.632812,False
15737888,850,Spain,Female,43,2,125510.820312,1,True,True,79084.101562,False
...,...,...,...,...,...,...,...,...,...,...,...
15606229,771,France,Male,39,5,0.000000,2,True,False,96270.640625,False
15569892,516,France,Male,35,10,57369.609375,1,True,True,101699.773438,False
15584532,709,France,Female,36,7,0.000000,1,False,True,42085.578125,True


In [12]:
## Query specific results
df_country = pd.read_sql(
      '''select country,count(*) as TOTAL_CUSTOMERS
from data
group by country
ORDER BY TOTAl_CUSTOMERS DESC''',
  engine)
df_country



,country,TOTAL_CUSTOMERS
0,France,5014
1,Germany,2509
2,Spain,2477



## using pyodbc


In [13]:

connection = pyodbc.connect('Trusted_Connection=yes', 
                     driver = '{ODBC Driver 17 for SQL Server}',
                     server = 'DABAS\SQLEXPRESS', 
                     database = 'ChurnDataset')


In [14]:
# get column names and type

query= "SELECT * FROM Data"
cursor = connection.cursor()
cursor.execute(query)
column_names = [desc[0] for desc in cursor.description]
column_types = [desc[1] for desc in cursor.description]
print(column_names)
print(column_types)



['customer_id', 'credit_score', 'country', 'gender', 'age', 'tenure', 'balance', 'products_number', 'credit_card', 'active_member', 'estimated_salary', 'churn']
[<class 'int'>, <class 'int'>, <class 'str'>, <class 'str'>, <class 'int'>, <class 'int'>, <class 'float'>, <class 'int'>, <class 'bool'>, <class 'bool'>, <class 'float'>, <class 'bool'>]


In [15]:
#combine these
for each in cursor.description:
    print(each[0:2])

('customer_id', <class 'int'>)
('credit_score', <class 'int'>)
('country', <class 'str'>)
('gender', <class 'str'>)
('age', <class 'int'>)
('tenure', <class 'int'>)
('balance', <class 'float'>)
('products_number', <class 'int'>)
('credit_card', <class 'bool'>)
('active_member', <class 'bool'>)
('estimated_salary', <class 'float'>)
('churn', <class 'bool'>)


In [18]:
#sinmpler and easier way of doing this
df_2 = pd.read_sql( "SELECT * FROM Data",connection)
df_2.dtypes

<ipython-input-18-6f3a47368e2f>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_2 = pd.read_sql( "SELECT * FROM Data",connection)


customer_id           int64
credit_score          int64
country              object
gender               object
age                   int64
tenure                int64
balance             float64
products_number       int64
credit_card            bool
active_member          bool
estimated_salary    float64
churn                  bool
dtype: object

In [6]:
#no of customers countrywise

df_pydoc = pd.read_sql( '''select country,count(*) as TOTAL_CUSTOMERS
from data
group by country
ORDER BY TOTAl_CUSTOMERS DESC''',connection)
df_pydoc


,country,TOTAL_CUSTOMERS
0,France,5014
1,Germany,2509
2,Spain,2477



Churn Analysis

In [14]:
#customers churned by country
Churn_df = pd.read_sql(
    '''SELECT country, 
    COUNT(CASE WHEN churn=1 THEN 1  END) AS Customers
    FROM Data
    GROUP BY country''',
    engine
)
Churn_df

,country,Customers
0,Germany,814
1,France,810
2,Spain,413


In [34]:
#customers churn rate from based on countrires
Churn_rate_countries_df = pd.read_sql(
'''SELECT 
    country,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN churn=1 THEN 1 ELSE 0 END) AS churned_customers,
    ROUND(SUM(CASE WHEN churn=1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*),2) AS churn_percentage
    FROM Data
    GROUP BY country'''
,engine
)
Churn_rate_countries_df


,country,total_customers,churned_customers,churn_percentage
0,Germany,2509,814,32.44
1,France,5014,810,16.15
2,Spain,2477,413,16.67


In [35]:
#churn rate genderwise

Chrun_rate_genderwise = pd.read_sql(
    '''SELECT 
    gender,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN churn=1 THEN 1 ELSE 0 END) AS churned_customers,
    ROUND(SUM(CASE WHEN churn=1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*),2) AS churn_percentage
    FROM Data
    GROUP BY gender''',
engine);
Chrun_rate_genderwise


,gender,total_customers,churned_customers,churn_percentage
0,Male,5457,898,16.46
1,Female,4543,1139,25.07


In [36]:
Chrun_rate_combined = pd.read_sql(
    '''SELECT 
    country,
    gender,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN churn=1 THEN 1 ELSE 0 END) AS churned_customers,
    ROUND(SUM(CASE WHEN churn=1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*),2) AS churn_percentage
    FROM Data
    GROUP BY country,gender''',
engine);
Chrun_rate_combined

,country,gender,total_customers,churned_customers,churn_percentage
0,Germany,Male,1316,366,27.81
1,Germany,Female,1193,448,37.55
2,Spain,Male,1388,182,13.11
3,Spain,Female,1089,231,21.21
4,France,Male,2753,350,12.71
5,France,Female,2261,460,20.34


In [46]:
Churn_age = '''WITH AgeGroupChurn AS (
    SELECT FLOOR(age / 10) * 10 AS Age_group, 
    (SUM(CASE WHEN churn = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) AS churn_rate
    FROM Data
    GROUP BY FLOOR(age / 10) * 10
)
SELECT * FROM AgeGroupChurn
ORDER BY AGE_group'''

Churn_age_df = pd.read_sql(Churn_age,engine)
Churn_age_df


,Age_group,churn_rate
0,10,6.122449
1,20,7.600503
2,30,10.883571
3,40,30.786860
4,50,56.041427
5,60,35.200000
6,70,10.294118
7,80,7.692308
8,90,0.000000
